# Universidad del Desarrollo
## Magister Data Science - Macroeconomía
### Grupo N°2: (Región de los Lagos)
* Jose Pedro
* Cristian
* Paul
* Max
* Hernan

## Calculo IPC

**Objetivo**

El objetivo de este notebook es el calculo de IPC según el siguiente algortimo.



$ 
\begin{equation*}
IPC_t   = |\sum_{k=1}^N \frac{p_i^t}{p_i^0} w_i| * 100
\end{equation*} 
$

$ 
\begin{equation*} 
p_i^t = \prod_{j=1}^n (\frac{v_j^t}{v_j^{t-1}})^{1/n} 
\end{equation*} 
$

Donde:
* $ p_i^t $ = Precio del producto i en el momento t
* $ p_i^0 $ = Precio del producto 0 en el mes base **(2014/08=100)**
* $ w_i   $ = Ponderación del producto i en la canasta
* $ v_j^t $ = Precio de la variaedad j en el momento t
* N = Numero de productos
* n = Numero de variedades

Dados los siguientes supuestos:
* n = 2
* t = 2014/09

Entonces:

P = ([ p_var1(2014-09) / p_var1(2014-08) ]$**$(1/2)) $*$ ([ p_var2(2014-09) / p_var2(2014-08) ]$**$(1/2))

In [1]:
import pandas as pd
import numpy as np
import calendar
import datetime

from dateutil.relativedelta import relativedelta

%matplotlib inline

### 1. Se carga RUT con los productos

In [2]:
df = pd.read_csv('output/canasta_ipc.csv')

In [3]:
df.head()

,Momento,Sector,Producto,Variedad,Promedio
0,2014-08,Frutillar,Aceite,Maravilla,1807.484158
1,2014-09,Frutillar,Aceite,Maravilla,1680.222200
2,2014-10,Frutillar,Aceite,Maravilla,1651.904700
3,2014-11,Frutillar,Aceite,Maravilla,1807.484158
4,2014-12,Frutillar,Aceite,Maravilla,1807.484158


In [4]:
df.Producto.value_counts()

Carnes       12692
Hortaliza     9587
Frutas        7951
Lacteos       2350
Pan           1925
Legumbre      1410
Cereales       940
Huevos         470
Harina         470
Aceite         470
Name: Producto, dtype: int64

### 2.- Transformamos Monento a un tipo de dato fecha para iterar sobre el periodo

In [5]:
df["Momento"] = pd.to_datetime(df["Momento"])

In [6]:
df.head()

,Momento,Sector,Producto,Variedad,Promedio
0,2014-08-01,Frutillar,Aceite,Maravilla,1807.484158
1,2014-09-01,Frutillar,Aceite,Maravilla,1680.222200
2,2014-10-01,Frutillar,Aceite,Maravilla,1651.904700
3,2014-11-01,Frutillar,Aceite,Maravilla,1807.484158
4,2014-12-01,Frutillar,Aceite,Maravilla,1807.484158


* Años base:

In [7]:
base = datetime.date(2014, 8, 1)

### 3. Creamos una funcion para restar un mes al periodo de estudio

In [8]:
def add_months(sourcedate, months):
    month = sourcedate.month - 1 + months
    year = sourcedate.year + month // 12
    month = month % 12 + 1
    day = min(sourcedate.day, calendar.monthrange(year, month)[1])
    return datetime.date(year, month, day)

### 4. Creamos función para calculo de la serie factores de producto

In [9]:
def GetSerieProducto(df_input, group):
    v = []
    for name, g in group: 
        
        #print(name)
        factor = 1
        n = g.Promedio.count()

        if g.Momento.iloc[0].date() > base:
            for row_num, row in g.iterrows(): 

                Pro1 = row["Promedio"]
                Mom2 = add_months(row["Momento"], -1)

                Pro2 = df_input[(df_input.Sector==row["Sector"]) & (df_input.Momento == Mom2) & (df_input.Variedad == row["Variedad"])]["Promedio"].iloc[0]

                factor *= (Pro1/Pro2)**(1/n)

        name = name + (factor,)  
        v.append(name)
    
    return pd.DataFrame(v, columns=['Momento', 'Sector', 'Producto', 'Factor'])

### 5. Aceite

In [10]:
df_aceite = df[df.Producto=="Aceite"]
group = df_aceite[df_aceite.Momento > "2015-03"].groupby(['Momento', 'Sector', 'Producto'])

In [11]:
dfs_aceite = GetSerieProducto(df_aceite, group)

In [12]:
dfs_aceite.sample(3)

,Momento,Sector,Producto,Factor
25,2015-09-01,Frutillar,Aceite,1.017865
139,2017-07-01,Puerto Varas,Aceite,1.016334
142,2017-08-01,Osorno,Aceite,1.021732


### 6. Harina

In [13]:
df_harina = df[df.Producto=="Harina"]
group = df_harina[df_harina.Momento > "2014-08"].groupby(['Momento', 'Sector', 'Producto'])

In [14]:
dfs_harina = GetSerieProducto(df_harina, group)

In [15]:
dfs_harina.sample(3)

,Momento,Sector,Producto,Factor
86,2016-02-01,Llanquihue,Harina,0.999800
192,2017-11-01,Osorno,Harina,0.990277
200,2018-01-01,Frutillar,Harina,1.030941


### 7. Cereales

In [16]:
df_cereales = df[df.Producto=="Cereales"]
group = df_cereales[df_cereales.Momento > "2014-08"].groupby(['Momento', 'Sector', 'Producto'])

In [17]:
dfs_cereales = GetSerieProducto(df_cereales, group)

In [18]:
dfs_cereales.sample(3)

,Momento,Sector,Producto,Factor
13,2014-11-01,Puerto Montt,Cereales,1.004182
158,2017-04-01,Puerto Montt,Cereales,1.016408
206,2018-02-01,Llanquihue,Cereales,1.007690


### 8. Azucar

In [19]:
df_azucar = df[df.Producto=="Azucar"]

In [27]:
df_azucar

,Momento,Sector,Producto,Variedad,Promedio


In [21]:
#df_azucar.isnull().sum()

In [22]:
#df_azucar["Variedad"].fillna("Azucar normal", inplace=True)

In [23]:
group = df_azucar[df_azucar.Momento > "2014-08"].groupby(['Momento', 'Sector', 'Producto'])

In [24]:
dfs_azucar = GetSerieProducto(df_azucar, group)

In [26]:
dfs_azucar

,Momento,Sector,Producto,Factor


### Creamos funcion para calculo del IPC